In [1]:
import os
import sys

In [2]:
%pwd

'c:\\Users\\justi\\Desktop\\MLOps Demo\\MLOPS-Demo-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\justi\\Desktop\\MLOps Demo\\MLOPS-Demo-Project'

In [5]:
sys.path.insert(0, '')


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [7]:
from src.mlops_project.constants import *
from src.mlops_project.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, 
               config_filepath = CONFIG_FILE_PATH, 
               params_filepath = PARAMS_FILE_PATH,
               schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from src.mlops_project import logger
from src.mlops_project.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloading with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-06-05 01:18:33,477: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-05 01:18:33,479: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-05 01:18:33,479: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-05 01:18:33,480: INFO: common: created directory at: artifacts]
[2024-06-05 01:18:33,481: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-05 01:18:33,760: INFO: 2522255329: artifacts/data_ingestion/data.zip downloading with following info: 
Connection: close
Content-Length: 53028
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "8049d9adcdb752e5fca65de90f9a04dc215596db947d69bb2bb3ccb42570153b"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7326:3037C6:6B0C9E:77957E:665FF528
Accept-Ranges: bytes
Date: Wed, 